In [1]:
# Adicionando ao path para importar funções do src
import sys

sys.path.append("../")


In [2]:
import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
import re
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import spacy
from wordcloud import WordCloud

from src.feature_extraction import get_chatgpt_sentiment_analysis

# Oculta warnings do notebook
# import warnings
# warnings.filterwarnings('ignore')

# Configurações de visualização
pd.set_option("display.max_columns", 100)
pd.options.display.float_format = "{:,.2f}".format
pd.options.display.max_colwidth = 300


In [3]:
dataset_path = "../data/processed/"
raw_df = pd.read_csv(f"{dataset_path}se_candidates_output_3.csv", sep=";")
print(raw_df.shape)
raw_df.head()


(1250, 79)


,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,CD_TIPO_ELEICAO,NM_TIPO_ELEICAO,NR_TURNO,CD_ELEICAO,DS_ELEICAO,DT_ELEICAO,TP_ABRANGENCIA,SG_UF,SG_UE,NM_UE,CD_CARGO,DS_CARGO,SQ_CANDIDATO,NR_CANDIDATO,NM_CANDIDATO,NM_URNA_CANDIDATO,NM_SOCIAL_CANDIDATO,NR_CPF_CANDIDATO,NM_EMAIL,CD_SITUACAO_CANDIDATURA,DS_SITUACAO_CANDIDATURA,CD_DETALHE_SITUACAO_CAND,DS_DETALHE_SITUACAO_CAND,TP_AGREMIACAO,NR_PARTIDO,SG_PARTIDO,NM_PARTIDO,NR_FEDERACAO,NM_FEDERACAO,SG_FEDERACAO,DS_COMPOSICAO_FEDERACAO,SQ_COLIGACAO,NM_COLIGACAO,DS_COMPOSICAO_COLIGACAO,CD_NACIONALIDADE,DS_NACIONALIDADE,SG_UF_NASCIMENTO,CD_MUNICIPIO_NASCIMENTO,NM_MUNICIPIO_NASCIMENTO,DT_NASCIMENTO,NR_IDADE_DATA_POSSE,NR_TITULO_ELEITORAL_CANDIDATO,CD_GENERO,DS_GENERO,CD_GRAU_INSTRUCAO,DS_GRAU_INSTRUCAO,CD_ESTADO_CIVIL,DS_ESTADO_CIVIL,CD_COR_RACA,DS_COR_RACA,CD_OCUPACAO,DS_OCUPACAO,VR_DESPESA_MAX_CAMPANHA,CD_SIT_TOT_TURNO,DS_SIT_TOT_TURNO,ST_REELEICAO,ST_DECLARAR_BENS,NR_PROTOCOLO_CANDIDATURA,NR_PROCESSO,CD_SITUACAO_CANDIDATO_PLEITO,DS_SITUACAO_CANDIDATO_PLEITO,CD_SITUACAO_CANDIDATO_URNA,DS_SITUACAO_CANDIDATO_URNA,ST_CANDIDATO_INSERIDO_URNA,NM_TIPO_DESTINACAO_VOTOS,CD_SITUACAO_CANDIDATO_TOT,DS_SITUACAO_CANDIDATO_TOT,ST_PREST_CONTAS,DS_URL,TW_USER,TW_followersCount,TW_friendsCount,TW_statusesCount,TW_favouritesCount,TW_listedCount,TW_mediaCount
0,10/04/2023,19:42:09,2022,2,ELEIÇÃO ORDINÁRIA,1,546,Eleições Gerais Estaduais 2022,02/10/2022,ESTADUAL,SP,SP,SÃO PAULO,6,DEPUTADO FEDERAL,250001620269,4066,NILTON LOURENÇO CANDIDO,DR NILTON CANDIDO,#NULO#,3068287820,NÃO DIVULGÁVEL,12,APTO,2,DEFERIDO,PARTIDO ISOLADO,40,PSB,PARTIDO SOCIALISTA BRASILEIRO,-1,#NULO#,#NULO#,#NULO#,250001682213,PARTIDO ISOLADO,PSB,1,BRASILEIRA NATA,SP,-3,CATANDUVA,28/06/1958,64.00,19179330124,2,MASCULINO,8,SUPERIOR COMPLETO,3,CASADO(A),1,BRANCA,131,ADVOGADO,"3,176,572.53",5,SUPLENTE,N,S,-1,6023622520226260000,2,DEFERIDO,2,DEFERIDO,SIM,Válido,2,Deferido,S,https://twitter.com/niltoncandido,niltoncandido,33,233,57,286,3,1
1,10/04/2023,19:42:09,2022,2,ELEIÇÃO ORDINÁRIA,1,546,Eleições Gerais Estaduais 2022,02/10/2022,ESTADUAL,SP,SP,SÃO PAULO,6,DEPUTADO FEDERAL,250001597710,3055,FERNANDO HOLIDAY SILVA BISPO,FERNANDO HOLIDAY,#NULO#,40850330858,NÃO DIVULGÁVEL,12,APTO,2,DEFERIDO,PARTIDO ISOLADO,30,NOVO,PARTIDO NOVO,-1,#NULO#,#NULO#,#NULO#,250001680945,PARTIDO ISOLADO,NOVO,1,BRASILEIRA NATA,SP,-3,SÃO PAULO,22/09/1996,26.00,398047820116,2,MASCULINO,8,SUPERIOR COMPLETO,1,SOLTEIRO(A),2,PRETA,278,VEREADOR,"3,176,572.53",5,SUPLENTE,N,S,-1,6003668920226260000,2,DEFERIDO,2,DEFERIDO,SIM,Válido,2,Deferido,S,https://www.twitter.com/fernandoholiday,fernandoholiday,468639,638,16962,15225,594,2792
2,10/04/2023,19:42:09,2022,2,ELEIÇÃO ORDINÁRIA,1,546,Eleições Gerais Estaduais 2022,02/10/2022,ESTADUAL,SP,SP,SÃO PAULO,7,DEPUTADO ESTADUAL,250001620597,40050,ELVIO FERNANDEZ GONZALEZ,ELVIO FERNANDEZ,#NULO#,13196728846,NÃO DIVULGÁVEL,12,APTO,2,DEFERIDO,PARTIDO ISOLADO,40,PSB,PARTIDO SOCIALISTA BRASILEIRO,-1,#NULO#,#NULO#,#NULO#,250001682228,PARTIDO ISOLADO,PSB,1,BRASILEIRA NATA,SP,-3,SÃO PAULO,09/03/1970,53.00,167516500175,2,MASCULINO,6,ENSINO MÉDIO COMPLETO,3,CASADO(A),1,BRANCA,536,TAXISTA,"1,270,629.01",5,SUPLENTE,N,S,-1,6024108120226260000,2,DEFERIDO,2,DEFERIDO,SIM,Válido,2,Deferido,S,https://www.twitter.com/TaxiZapElite,taxizapelite,902,1719,3685,349,2,768
3,10/04/2023,19:42:09,2022,2,ELEIÇÃO ORDINÁRIA,1,546,Eleições Gerais Estaduais 2022,02/10/2022,ESTADUAL,SP,SP,SÃO PAULO,7,DEPUTADO ESTADUAL,250001611440,23230,RODRIGO CAPEL,RODRIGO CAPEL,#NULO#,28302795810,NÃO DIVULGÁVEL,12,APTO,2,DEFERIDO,FEDERAÇÃO,23,CIDADANIA,CIDADANIA,1,Federação PSDB Cidadania,PSDB/CIDADANIA,PSDB/CIDADANIA,250001681575,FEDERAÇÃO,PSDB/CIDADANIA,1,BRASILEIRA NATA,SP,-3,DIADEMA,01/07/1979,43.00,222498950116,2,MASCULINO,8,SUPERIOR COMPLETO,1,SOLTEIRO(A),1,BRANCA,131,ADVOGADO,"1,270,629.01",5,SUPLENTE,N,S,-1,6016036120226260000,2,DEFERIDO,2,DEFERIDO,SIM,Válido,2,Deferido,S,"https://mobile.twitter.com/RodrigoCapelDr,",rodrigocapeldr,0,12,2,2,0,1
4,10/04/2023,19:42:09,2022,2,ELEIÇÃO ORDINÁRIA

In [4]:
columns = [
    "NM_UE",
    "DS_CARGO",
    "SQ_CANDIDATO",
    "NM_URNA_CANDIDATO",
    "SG_PARTIDO",
    "DS_SIT_TOT_TURNO",
    "DS_URL",
    "TW_USER",
    "TW_followersCount",
    "TW_friendsCount",
    "TW_statusesCount",
    "TW_favouritesCount",
    "TW_listedCount",
    "TW_mediaCount",
]
df = raw_df[columns]
df["SQ_CANDIDATO"] = df["SQ_CANDIDATO"].astype("object")
df.head()


/tmp/ipykernel_3431/2412712231.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['SQ_CANDIDATO'] = df['SQ_CANDIDATO'].astype('object')


,NM_UE,DS_CARGO,SQ_CANDIDATO,NM_URNA_CANDIDATO,SG_PARTIDO,DS_SIT_TOT_TURNO,DS_URL,TW_USER,TW_followersCount,TW_friendsCount,TW_statusesCount,TW_favouritesCount,TW_listedCount,TW_mediaCount
0,SÃO PAULO,DEPUTADO FEDERAL,250001620269,DR NILTON CANDIDO,PSB,SUPLENTE,https://twitter.com/niltoncandido,niltoncandido,33,233,57,286,3,1
1,SÃO PAULO,DEPUTADO FEDERAL,250001597710,FERNANDO HOLIDAY,NOVO,SUPLENTE,https://www.twitter.com/fernandoholiday,fernandoholiday,468639,638,16962,15225,594,2792
2,SÃO PAULO,DEPUTADO ESTADUAL,250001620597,ELVIO FERNANDEZ,PSB,SUPLENTE,https://www.twitter.com/TaxiZapElite,taxizapelite,902,1719,3685,349,2,768
3,SÃO PAULO,DEPUTADO ESTADUAL,250001611440,RODRIGO CAPEL,CIDADANIA,SUPLENTE,"https://mobile.twitter.com/RodrigoCapelDr,",rodrigocapeldr,0,12,2,2,0,1
4,SÃO PAULO,DEPUTADO ESTADUAL,250001614933,BOMBEIRO TENENTE LESSA,PTB,NÃO ELEITO,https://twitter.com/LessaTenente?t=nqNj8fAySbh5T-uZAnwrfA&s=08,lessatenente,49,135,49,106,0,30


In [5]:
df.describe()


,TW_followersCount,TW_friendsCount,TW_statusesCount,TW_favouritesCount,TW_listedCount,TW_mediaCount
count,"1,250.00","1,250.00","1,250.00","1,250.00","1,250.00","1,250.00"
mean,"41,559.77",937.40,"4,320.44","4,580.29",79.29,626.92
std,"224,527.50","4,738.90","9,603.65","16,632.07",302.14,"1,527.97"
min,0.00,0.00,0.00,0.00,0.00,0.00
25%,15.00,27.00,33.00,9.00,0.00,6.00
50%,307.00,179.00,511.00,218.50,2.00,83.50
75%,"3,271.75",615.75,"4,051.00","2,127.75",31.00,539.75
max,"2,853,478.00","109,796.00","102,974.00","257,023.00","4,731.00","25,500.00"


In [6]:
df.groupby("DS_SIT_TOT_TURNO").describe()


TW_followersCount                                             \
                             count       mean          std      min       25%   
DS_SIT_TOT_TURNO                                                                
#NULO#                       21.00  36,245.10   113,674.21     0.00      8.00   
2º TURNO                      2.00   6,289.50     2,443.05 4,562.00  5,425.75   
ELEITO                        9.00 876,991.89 1,004,763.24     0.00 71,697.00   
ELEITO POR MÉDIA             35.00  51,886.03   105,516.50     0.00    549.00   
ELEITO POR QP               171.00 127,244.40   405,204.27     0.00  1,315.00   
NÃO ELEITO                  267.00  38,829.94   238,354.52     0.00      6.00   
SUPLENTE                    745.00  12,537.81    64,069.72     0.00     10.00   

                                                      TW_friendsCount  \
                        50%          75%          max           count   
DS_SIT_TOT_TURNO                                                        
#NULO#                41.00       186.00   392,539.00           21.00   
2º TURNO           6,289.50     7,153.25     8,017.00            2.00   
ELEITO           530,124.00 1,273,556.00 2,523,954.00            9.00   
ELEITO POR MÉDIA   2,895.00    15,356.50   372,034.00           35.00   
ELEITO POR QP      6,838.00    35,629.00 2,563,232.00          171.00   
NÃO ELEITO           243.00     2,184.00 2,853,478.00          267.00   
SUPLENTE             127.00     1,265.00   943,316.00          745.00   

                                                                             \
                     mean       std   min    25%    50%      75%        max   
DS_SIT_TOT_TURNO                                                              
#NULO#             423.24  1,083.09  0.00  12.00  47.00   331.00   4,985.00   
2º TURNO           427.00    492.15 79.00 253.00 427.00   601.00     775.00   
ELEITO             337.33    266.94  0.00 146.00 331.00   597.00     733.00   
ELEITO POR MÉDIA 5,902.49 16,463.24  0.00 228.00 554.00 1,607.00  70,328.00   
ELEITO POR QP    1,602.71  8,548.97  0.00  92.00 342.00 1,018.00 109,796.00   
NÃO ELEITO         890.53  3,385.79  0.00  13.50 160.00   525.00  36,297.00   
SUPLENTE           591.34  1,757.52  0.00  21.00 140.00   541.00  29,034.00   

                 TW_statusesCount                                       \
                            count     mean       std      min      25%   
DS_SIT_TOT_TURNO                                                         
#NULO#                      21.00 1,686.19  3,805.30     0.00    11.00   
2º TURNO                     2.00 2,764.00  2,257.08 1,168.00 1,966.00   
ELEITO                       9.00 8,278.89  7,497.53     0.00 3,828.00   
ELEITO POR MÉDIA            35.00 5,765.31  7,597.51     0.00   287.50   
ELEITO POR QP              171.00 8,665.27 12,489.31     0.00   699.00   
NÃO ELEITO                 267.00 3,955.88 10,517.61     0.00    15.50   
SUPLENTE                   745.00 3,416.55  8,385.01     0.00    25.00   

                                               TW_favouritesCount           \
                      50%       75%        max              count     mean   
DS_SIT_TOT_TURNO                                                             
#NULO#              54.00  1,280.00  16,899.00              21.00 2,674.48   
2º TURNO         2,764.00  3,562.00   4,360.00               2.00 1,146.50   
ELEITO           7,846.00 10,613.00  24,748.00               9.00 6,897.44   
ELEITO POR MÉDIA 2,515.00  8,451.00  31,799.00              35.00 3,787.51   
ELEITO POR QP    4,322.00 12,431.00 102,974.00             171.00 8,660.81   
NÃO ELEITO         394.00  2,452.50  83,582.00             267.00 6,194.70   
SUPLENTE           311.00  2,414.00  76,729.00             745.00 3,137.30   

                                                                       \
                       std    min    25%      50%      75%        max   
DS_SIT_TOT_TURNO

In [7]:
elected_values = ["ELEITO POR QP", "ELEITO POR MÉDIA", "ELEITO"]
non_elected_values = ["#NULO#", "2º TURNO", "NÃO ELEITO", "SUPLENTE"]
pd.set_option("display.max_rows", 300)


In [8]:
# Deputados Federais Eleitos em SP
sample_df = df[(df["NM_UE"] == "SÃO PAULO") & (df["DS_CARGO"] == "DEPUTADO FEDERAL")]
sample_df = sample_df.loc[(sample_df["DS_SIT_TOT_TURNO"].isin(elected_values))]

sample_df.sort_values("TW_followersCount", ascending=False)["TW_USER"].values


array(['bolsonarosp', 'zambelli2210', 'guilhermeboulos', 'rsallesmma',
       'marcofeliciano', 'samiabomfim', 'luizaerundina',
       'pauloteixeira13', 'erikakhilton', 'guajajarasonia', 'rfalcao13',
       'carlossampaio_', 'carloszarattini', 'tiriricanaweb',
       'del_brunolima', 'rosangelamorosp', 'baleia_rossi',
       'alencarbraga13', 'marcospereira04', 'adriventurasp',
       'jilmartatto', 'niltotatto', 'luizmarinhopt', 'felipebecari',
       'pauloabarbosa', 'alexmanente23', 'capitaoaugusto_', 'julianapt',
       'deprosanavalle', 'arnaldojardim', 'depjefferson',
       'df_paulofreire', 'marcobertaiolli', 'kikoceleguim',
       'jonasdonizette_', 'ricardosilva', 'falecomvinicius',
       'davidbrsoares', 'mauricioneves_', 'capitaoderrite', 'acrpolitico',
       'silva_marina'], dtype=object)

In [9]:
# Deputados Federais Não-eleitos em SP
sample_df = df[(df["NM_UE"] == "SÃO PAULO") & (df["DS_CARGO"] == "DEPUTADO FEDERAL")]
sample_df = sample_df.loc[(sample_df["DS_SIT_TOT_TURNO"].isin(non_elected_values))]

sample_df.sort_values("TW_followersCount", ascending=False)["TW_USER"].values


array(['abrahamweint', 'arthurweint', 'augustodeab', 'fernandoholiday',
       'ivanvalente', 'douglasgarcia', 'joicehasselmann', 'thiagoresiste',
       'camargodireita', 'aragaomosart', 'hoc111', 'ariadnaarantes',
       'pablomarcal', 'ricardogalvaosp', 'andrepedrosobr', 'coroneltadeu',
       'tandaracaixeta', 'daianemels', 'flavinhocn', 'diogoforjaz',
       'isapenna', 'danieljosebr', 'wesleyros', 'guigapeixotosp',
       'negrobelchior', 'vicentinhopt', 'coroneltelhada', 'soutamires_sp',
       'janainalimasp', 'alexisfonteyne', 'antonionetopdt',
       'safatle_oficial', 'luismirandausa', 'vaccarezza',
       'marinahelenabr', 'davilafrederico', 'coronelhomerosp',
       'rodrigoagost', 'jose_anibal', 'katiasastre', 'eduardo_cury',
       'andreiatofollo', 'rossieli', 'drviniciussp', 'cardosojrdavid',
       'adeoliveirabr', 'tourinhopedro', 'joseamerico_sp', 'satofabio',
       'morosenbergsp', 'rogeriovtr', 'agsampaio', 'larratsymmy',
       'denisandia', 'neziomonteiro', 'ce

In [10]:
# Deputados Federais Eleitos em RJ
sample_df = df[
    (df["NM_UE"] == "RIO DE JANEIRO") & (df["DS_CARGO"] == "DEPUTADO FEDERAL")
]
sample_df = sample_df.loc[(sample_df["DS_SIT_TOT_TURNO"].isin(elected_values))]

sample_df.sort_values("TW_followersCount", ascending=False)["TW_USER"].values


array(['carlosjordy', 'depheliolopes', 'glauber_braga', 'delegadoramagem',
       'taliriapetrone', 'dasilvabenedita', 'pazuellogeneral',
       'mottatarcisio', 'oficialluizlima', 'toniettochris', 'depsostenes',
       'danielsoranz', 'dephugoleal', 'pedropaulo', 'altineu',
       'rjlauracarneiro', 'rosangelasgomes', 'juninhodopneu',
       'lucianovieirarj', 'chico_psol', 'danielawaguinho', 'otonidepaula',
       'sgt_portugal', '1384588949797093381'], dtype=object)

In [11]:
# Deputados Federais Eleitos em MG
sample_df = df[(df["NM_UE"] == "MINAS GERAIS") & (df["DS_CARGO"] == "DEPUTADO FEDERAL")]
sample_df = sample_df.loc[(sample_df["DS_SIT_TOT_TURNO"].isin(elected_values))]

sample_df.sort_values("TW_followersCount", ascending=False)["TW_USER"].values


array(['nikolas_dm', 'andrejanonesadv', 'cabojunioamaral', 'dudasalabert',
       'erosbiondini', 'marceloalvaroan', 'celiaxakriaba',
       'dep_padrejoao', 'deppauloguedes', 'lincoln_portela',
       'mario_heringer', 'luistibeoficial', 'depleomonteiro',
       'fredcostadep', 'odaircunhamg', 'anapimentel_jf',
       'domingossaviomg', 'pauloabiackel', 'drfredericomg', 'apjunqueira',
       'lafayetteandrad', 'euclydespetter', 'greyceelias', 'zevitor_mg',
       'delegadaione', 'miguelangelo', 'varellamisael'], dtype=object)

In [14]:
# Deputados Federais Eleitos em ES
sample_df = df[
    (df["NM_UE"] == "ESPÍRITO SANTO") & (df["DS_CARGO"] == "DEPUTADO FEDERAL")
]
sample_df = sample_df.loc[(sample_df["DS_SIT_TOT_TURNO"].isin(elected_values))]

sample_df.sort_values("TW_followersCount", ascending=False)["TW_USER"].values


array(['evairdemelo', 'gilvandafederal', 'deputadofoletto'], dtype=object)

In [15]:
# Deputados Estaduais Eleitos em RJ
sample_df = df[
    (df["NM_UE"] == "RIO DE JANEIRO") & (df["DS_CARGO"] == "DEPUTADO ESTADUAL")
]
sample_df = sample_df.loc[(sample_df["DS_SIT_TOT_TURNO"].isin(elected_values))]

sample_df.sort_values("TW_followersCount", ascending=False)["TW_USER"].values


array(['alanlopesrio', 'minc_rj', 'renatasouzario', 'deputadomoraes',
       'depmgualberto', 'danimontpsol', 'serafinipsol', 'yurimourarj',
       'delmartharocha', 'deputadopoubel', 'samuelmalafaia',
       'rjrodrigoamorim', 'luizpaulo_dep', 'drserginho_',
       'depandrecorrea', 'cavaliererj', 'gustavotutuca',
       'bacellaroficial', 'machadocarlasjb', 'gi_monteirorj',
       'andrezinho_cec', 'pedroricardo_rj', 'depceliajordao',
       'eusoujuliorj'], dtype=object)

In [16]:
# Deputados Estaduais Eleitos em MG
sample_df = df[
    (df["NM_UE"] == "MINAS GERAIS") & (df["DS_CARGO"] == "DEPUTADO ESTADUAL")
]
sample_df = sample_df.loc[(sample_df["DS_SIT_TOT_TURNO"].isin(elected_values))]

sample_df.sort_values("TW_followersCount", ascending=False)["TW_USER"].values


array(['mariocaixa', 'brunoenglerdm', 'joaovitorxavier', 'bellagoncalvs',
       'andreiadejesuus', 'macaeevaristo', 'biacerqueira_',
       'eduardoazevemg', 'caporezzodm', 'rafaelmartinstv',
       'depsgtrodrigues', 'depsgtrodrigues', 'aleportela', 'leninhamoc13',
       'depulyssesgomes', 'depcelinho', 'cassioafsoares',
       'depprofcleiton', 'doorgalandrada', 'wendelprofessor',
       'dep_anapaula18', 'delegadasheila', 'deputadolemos',
       'lelecopimentel_', 'deputadocarlosh', 'rodrigolopes_mg',
       'ionepinheiro9', 'gregoporminas'], dtype=object)

In [17]:
# Deputados Estaduais Eleitos em ES
sample_df = df[
    (df["NM_UE"] == "ESPÍRITO SANTO") & (df["DS_CARGO"] == "DEPUTADO ESTADUAL")
]
sample_df = sample_df.loc[(sample_df["DS_SIT_TOT_TURNO"].isin(elected_values))]

sample_df.sort_values("TW_followersCount", ascending=False)["TW_USER"].values


array(['camilavaladao50', 'sergiomenegueli', 'darypagung',
       'depjanetedesa', 'fgandini1', 'tyago_hoffmann', 'scaramussalucas',
       'raquellessa11'], dtype=object)

In [20]:
# Senadores Eleitos e Não-eleitos em SP, MG, RJ e ES
sample_df = df[(df["DS_CARGO"] == "SENADOR")]
sample_df.sort_values("TW_followersCount", ascending=False)["TW_USER"].values


array(['romarioonze', 'magnomalta', 'astro_pontes', 'alessandromolon',
       'cabodaciolo', 'cleitinhotmj', 'marciofrancasp', 'aldorebelo',
       'senadorarose', 'andrececiliano', 'ricardomellaosp', 'asilveiramg',
       'edsonaparecido', 'sarapsol', 'titobellini', 'brunomirandasal',
       'mussoerick', 'hiranroedel', 'drpaulomarcelo1', 'suedhaidar3535',
       'skiterfilipe', 'manchapstu', 'barbarasinedinocom'], dtype=object)

In [21]:
# Governadores Eleitos e Não-eleitos em SP, MG, RJ e ES
sample_df = df[(df["DS_CARGO"] == "GOVERNADOR")]
sample_df.sort_values("TW_followersCount", ascending=False)["TW_USER"].values


array(['haddad_fernando', 'tarcisiogdf', 'marcelofreixo',
       'alexandrekalil', 'romeuzema', 'wilsonwitzel', 'viniciuspoit',
       'casagrande_es', 'pauloganime', 'claudiocastrorj', 'carlosaviana',
       '_rodrigoneves_', 'manato_es', 'altinopstu', 'serraeduardopcb',
       'renatareginapcb', 'lorene_psol', 'vanessapstubh', 'rodrigogarcia',
       'zanonguerino', 'aridelmo_br', 'cabotristao', 'pmcassiosousa',
       'marcus_pestana', 'indira_up', 'twittercyro16com'], dtype=object)